In [57]:
import os



In [58]:
pwd

'/home/kuldeep-chamoli/Desktop/Mtech/krish_naik/MLOPS/datascipro'

In [59]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [60]:
from src.datascipro.constants import  *
from src.datascipro.utils.common import read_yaml,create_directories


In [61]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        """
        Loads configuration, parameters, and schema YAML files.
        Also creates the root artifacts directory.
        """
        self.config = read_yaml(config_filepath)   # Load main config
        self.params = read_yaml(params_filepath)   # Load model/experiment params
        self.schema = read_yaml(schema_filepath)   # Load data schema

        # Ensure the artifacts root directory exists
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Extracts the data ingestion-related config section and returns
        a DataIngestionConfig dataclass object.
        """
        config = self.config.data_ingestion
        create_directories([config.root_dir])  # Create dir for data ingestion artifacts

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [62]:
import os
import urllib.request as request
from src.datascipro import logger
import zipfile

In [67]:
import os
import zipfile
from urllib import request
from src.datascipro import logger

class DataIngestion:
    def __init__(self, config: DataIngestionconfig):
        self.config = config

    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info: \n{headers}")
        else:
            logger.info("File already exists")

    # Extracting the downloaded zip file
    def extract_zip_file(self):
        """
        Extracts the zip file into the specified unzip directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted zip file to: {unzip_path}")


In [68]:
pwd

'/home/kuldeep-chamoli/Desktop/Mtech/krish_naik/MLOPS/datascipro'

In [69]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-09 13:51:58,913:INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-04-09 13:51:58,915:INFO: common: YAML file: params.yaml loaded successfully]
[2025-04-09 13:51:58,921:INFO: common: YAML file: schema.yaml loaded successfully]
[2025-04-09 13:51:58,927:INFO: common: Created directory at: artifacts]
[2025-04-09 13:51:58,929:INFO: common: Created directory at: artifacts/data_ingestion]
[2025-04-09 13:51:59,565:INFO: 1140020354: artifacts/data_ingestion/data.zip downloaded with the following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: CC7C:15C22F:804D6:16F879:67F62D52
Accept-Ranges: bytes
Date: Wed, 09 A

In [64]:
import os
print(os.getcwd())


/home/kuldeep-chamoli/Desktop/Mtech/krish_naik/MLOPS/datascipro
